In [1]:
import sys
sys.path.insert(0, '../lib')

In [2]:
import collections
import itertools
import functools
import os
import pathlib
import shutil

import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.decomposition
import plotly.express as px
import scipy
import statsmodels.stats.multitest

import common_data
import factor_interpretation as fi

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
%config InlineBackend.figure_format = "retina"
# mpl.rcParams['figure.constrained_layout.use'] = True

In [5]:
from IPython.display import display, HTML
display(HTML("""
<style>
.lm-Widget {overflow-x: scroll !important;}
</style>
"""))

# Pathogen pseudobulk principal component analysis

## Methods:
To find differentially-expressed genes between pathogen groups (7) and control groups (2), we use pseudobulk approach with certain modification, detailed below. Pseudobulk approach shows better agreement with bulk approach during benchmarking (Squair et al., Nat Commun, 2021) compared to tests that do not account for cells’ belonging to different samples, and does not suffer from inflated p-values.

First, we prepared pseudobulk samples from our single-cell data. For each annotated cell type $T$, we excluded samples that didn’t have 50 cells (determined in [this notebook](https://github.com/NUPulmonary/serniczek/blob/main/05_pseudobulk/01_pseudobulk_number.ipynb)). Each sample belongs to strictly one pathogen or control group $G$. Next, we excluded genes that are not consistently expressed in the data from differential gene expression analysis: if a gene was detected (count > 0) in less than 80% of pseudobulk samples after filtering from sample group $G$, we excluded that gene. We also excluded 
* mitochondrial genes
* ribosomal genes
* genes with certain name patterns (see [here](https://www.biostars.org/p/9553891/))

For full gene exclusion code see: [\[1\]](https://github.com/NUPulmonary/serniczek/blob/main/02b_integration/10_raw-object.ipynb), [\[2\]](https://github.com/NUPulmonary/serniczek/blob/main/04b_geneformer/05a_gene_filtering_pseudobulk.ipynb).

Then we summed up raw gene counts for all cells in that sample that belong to the annotated cell type $T$ (see [here](https://github.com/NUPulmonary/serniczek/blob/main/05_pseudobulk/10_pathogen_pseudobulk.ipynb)).
Next, we constructed DESeq2 object will all pathogen and control groups (9 total) together with model expression `~ group + sex` and fitType set to `'local'`. Here we excluded samples that do not have clearly defined pathogen group. Then we applied DESeq2 [variance-stabilizing transformation](https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#count-data-transformations) with `blind=FALSE` to obtain normalized pseudobulk counts. Code is in [this notebook](https://github.com/NUPulmonary/serniczek/blob/main/05_pseudobulk/11_pathogen_deseq2.R).

In this notebook we proceed with Principal Component analysis of these DESeq2 normalized counts individually per cell type.

For each cell type we selected top 2000 genes based on their variance across all pseudobulk samples with existing pathogen group label. Then we applied PCA to the resulting normalized count matrix, and plotted explained variance by principle component, and PC1 vs PC2 plots, and saved gene loadings for top 10 PCs. To test association of the first 10 PCs with any of the sample covariates, we ran statistical tests separately for categorical and numerical covariates. For categorical covariates we first filled in some missing values for healthy control samples (full list [here](https://github.com/NUPulmonary/serniczek/blob/main/lib/common_data.py#L33)), then excluded remaining missing values and assessed the number of categories left. For 2 categories we ran t-test, for more than 2 categories we ran ANOVA on PC values for samples from different groups, and obtained corresponding p-values. For numerical covariates, we excluded missing values and ran Pearson correlation, and obtained corresponding p-values and correlation coefficients. Next we ran FDR correction for all tests (10 PCs × number of categorical/numerical covariates) in categorical or numerical group independently. FDR q-values below 0.05 were annotated as significant. For numerical covariates we plot correlation coefficient and mark significant tests with a star. For categorical covariates we plot $-log_{10}($q-value$)$. Rows and columns of the heatmap are hierarchically clustered based on euclidean distance using Ward linkage method.

## Goals:

1. For each cell type
   1. Select top $N$ variable features (per DESeq2, maybe not needed)
   2. Run PCA
   3. Save & plot elbow plot
   4. Save loadings
   5. Plot interactive PC with plotly
   6. Correlate to $P$ PCs with numerical features + run ANOVA for pathogen categories
   7. Run ANOVA for multiple levels of pathogens

In [6]:
ROOT = pathlib.Path('/projects/b1196/ewa_group/serniczek/data')
BASE = ROOT / '05_pseudobulk/10_pathogen'

In [7]:
TOP_N_GENES = 2000
TOP_N_PCS = 10

In [8]:
# ['SARS-CoV-2; Gram+', 'Pseudomonas aeruginosa; SARS-CoV-2',
# 'SARS-CoV-2', 'Gram+', 'NPC', 'Gram-*', 'Healthy', 'Gram-*; Gram+',
# 'Pseudomonas aeruginosa'
PALETTE = {
    'Healthy': "#5cd061", # green
    'NPC': "#256b33", # dark green
    'SARS-CoV-2' : "#e51d23", # red
    'SARS-CoV-2; Gram+' : "#ffc326", # gold
    # 'discard': "#cfcfcf", # grey
    'Pseudomonas aeruginosa; SARS-CoV-2': "#9858d6", # purple
    'Pseudomonas; SARS-CoV-2': "#9858d6", # purple
    'Gram+': "#013265", # blue
    'Pseudomonas aeruginosa': "#fb4e93", # pink
    'Pseudomonas': "#fb4e93", # pink
    'Gram-*; Gram+': "#1ceaf9", # cyan
    'Gram-*': "#770c2e" # dark red
}
ORDER = [
    'Healthy', 'NPC', 'SARS-CoV-2', 'Gram+', 'Gram-*', 'Pseudomonas', 
    'SARS-CoV-2; Gram+', 'Pseudomonas; SARS-CoV-2', 'Gram-*; Gram+'
]

In [9]:
sc_meta = pd.read_csv(
    ROOT / '02b_integration/09_final_full-1/09_final_full-1-metadata.csv',
    index_col=0
)

/tmp/ipykernel_50933/4000152471.py:1: DtypeWarning: Columns (27,28,32,33,36,37,46,50,52,54,55,56,63) have mixed types. Specify dtype option on import or set low_memory=False.
  sc_meta = pd.read_csv(


In [10]:
pseudobulk_n_cells = sc_meta.groupby(['individual', 'Level_6']).size().reset_index(name='n_cells')

In [11]:
pseudobulk_n_cells = pseudobulk_n_cells.loc[pseudobulk_n_cells.n_cells.ge(50)].copy()

In [12]:
sample_to_group = sc_meta.groupby('individual').head(1).set_index('individual').perturbation_groups

In [13]:
pseudobulk_n_cells['group'] = sample_to_group[pseudobulk_n_cells.individual].values

In [14]:
pseudobulk_n_cells = pseudobulk_n_cells.loc[pseudobulk_n_cells.group.ne('discard')].copy()

In [15]:
pseudobulk_n_cells.rename(columns={'individual': 'sample', 'Level_6': 'cell_type'}, inplace=True)

In [16]:
sample_meta = sc_meta.groupby('individual').head(1)[
    ['days_on_ventilator', 'sequencing_depth', 'sequencing_saturation',
    'frac_reads_in_cells', 'viability', 'individual']
].set_index('individual')

In [17]:
categorical_covariates = common_data.get_sc_categorical_covariates()

In [18]:
numerical_covariates = common_data.get_sc_numerical_covariates()

In [19]:
class CellTypeInfo:
    def __init__(self, path, pseudobulk_n_cells):
        self.path = path
        self.comparisons = []
        self.meta = pd.read_csv(path / 'data/meta.csv', index_col=0)
        self.name = self.meta.cell_type.values[0]

        pseudobulk_info = pseudobulk_n_cells.loc[pseudobulk_n_cells.cell_type.eq(self.name)]
        self.load_comparisons(pseudobulk_info)

        self.vst = pd.read_table(path / '_deseq2/transformed.tsv', delim_whitespace=True).T
        self.run_pca()

    def load_comparisons(self, pseudobulk_info):
        for run in self.path.glob('**/_deseq2.csv'):
            info = pd.read_csv(run).iloc[0]
            self.comparisons.append(run.parent / 'degs.csv')

    def select_top_genes(self):
        top_genes = self.vst.var(axis=0).sort_values().index[-TOP_N_GENES:]
        return self.vst.loc[:, top_genes].copy()

    def run_pca(self):
        self.top_vst = self.select_top_genes()
        self.pca = sklearn.decomposition.PCA()
        self.pcs = self.pca.fit_transform(self.top_vst)

    def plot_pca_elbow(self):
        to_plot = min(50, self.pca.n_components_)
        fig, ax = plt.subplots(figsize=(6, 3), constrained_layout=True)
        ax.bar(range(to_plot), self.pca.explained_variance_ratio_[:to_plot] * 100)
        ax.set_xlabel('PC', size=14)
        ax.set_ylabel('% variance explained\n(top 2000 genes)', size=14)
        ax.set_title(self.name, size=16)
        return fig

    def get_pc_weights(self):
        return pd.DataFrame(
            self.pca.components_[:TOP_N_PCS].T, 
            index=self.top_vst.columns,
            columns='PC' + (pd.Series(range(TOP_N_PCS)) + 1).astype(str)
        ).sort_values('PC1')

    def get_pc_df(self, additional_meta):
        pc_df = self.meta.copy()
        pc_df.loc[:, 'PC' + (pd.Series(range(TOP_N_PCS)) + 1).astype(str)] = self.pcs[:, :TOP_N_PCS]
        pc_df.group = pc_df.group.str.replace('Pseudomonas aeruginosa', 'Pseudomonas')
        pc_df.loc[:, additional_meta.columns] = additional_meta.loc[pc_df['sample'].values].values
        pc_df['size'] = pc_df.days_on_ventilator.copy()
        pc_df.loc[pc_df['size'].lt(0), 'size'] = 0
        pc_df['size'] = (pc_df['size'] + 0.5) ** 0.5
        return pc_df

    @property
    def n_comparisons(self):
        return len(self.comparisons)

In [20]:
%%time
data = []
for cell_type_path in sorted(BASE.iterdir()):
    if not (cell_type_path / '_deseq2/transformed.tsv').exists():
        continue
    info = CellTypeInfo(cell_type_path, pseudobulk_n_cells)
    if info.n_comparisons > 0:
        data.append(info)

CPU times: user 2min 4s, sys: 1min 48s, total: 3min 53s
Wall time: 13.4 s


In [21]:
[i.pca.n_components_ for i in data]

[162, 173, 48, 56, 153, 105, 184, 184, 10, 166, 22, 75, 53, 69, 71, 106]

In [22]:
def plot_pc(cell_type, pc_df):
    return px.scatter(
        pc_df, 
        x='PC1', 
        y='PC2', 
        color='group',
        size='size', 
        hover_data=dict(
            sample=True, 
            days_on_ventilator=True,
            size=False,
            sex=True
        ),
        color_discrete_map=PALETTE,
        labels=dict(
            PC1=f'PC1 {cell_type.pca.explained_variance_ratio_[0] * 100:.1f}%',
            PC2=f'PC2 {cell_type.pca.explained_variance_ratio_[1] * 100:.1f}%',
        ),
        category_orders=dict(
            group=ORDER
        ),
        title=cell_type.name
    )

In [23]:
def plot_pc_cat_assoc(pc_corrs):
    pc_corrs = pc_corrs.copy()
    pc_corrs[pc_corrs == 0] = 10**-15
    pc_annot = pd.DataFrame(index=pc_corrs.index, columns=pc_corrs.columns)
    pc_annot[-np.log10(pc_corrs) > -np.log10(0.05)] = '*'
    pc_annot[pc_annot != '*'] = ''
    cg = sns.clustermap(
        -np.log10(pc_corrs).T,
        method='ward',
        cmap='Blues',
        annot=pc_annot.T,
        fmt='s',
        cbar_kws=dict(
            label='$-log10($p-value$)$'
        ),
        dendrogram_ratio=0.1
    )
    cg.ax_col_dendrogram.set_title(cell_type.name, size=16)
    cg.fig.subplots_adjust(top=0.95)
    cg.ax_cbar.set_position((0.02, 0.85, 0.02, 0.1))
    return cg

In [24]:
def plot_pc_corrs(pc_corrs, pc_corr_pvals):
    pc_annot = pd.DataFrame(index=pc_corr_pvals.index, columns=pc_corr_pvals.columns)
    pc_annot[-np.log10(pc_corr_pvals) > -np.log10(0.05)] = '*'
    pc_annot[pc_annot != '*'] = ''
    # print(pc_annot.to_numpy())
    cg = sns.clustermap(
        pc_corrs.T,
        method='ward',
        cmap='vlag',
        annot=pc_annot.T,
        fmt='s',
        cbar_kws=dict(
            label='Pearson $r$'
        ),
        dendrogram_ratio=0.1,
        center=0,
    )
    cg.ax_col_dendrogram.set_title(cell_type.name, size=16)
    cg.fig.subplots_adjust(top=0.95)
    cg.ax_cbar.set_position((0.02, 0.85, 0.02, 0.1))
    return cg

In [25]:
def plot_pc_cat_boxplot(name, pc_df, pc, cat, na_values):
    if cat == 'perturbation_groups':
        pc_df[cat] = pc_df[cat].cat.rename_categories({
            'Pseudomonas aeruginosa': 'Pseudomonas',
            'Pseudomonas aeruginosa; SARS-CoV-2': 'Pseudomonas; SARS-CoV-2'
        })
    
    fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)
    stats_results = []
    na_value = na_values.get(cat)
    for d1, d2 in itertools.combinations(pc_df[cat].unique(), 2):
        if na_value and na_value in (d1, d2):
            continue
        days1 = pc_df[pc][pc_df[cat].eq(d1)].dropna()
        days2 = pc_df[pc][pc_df[cat].eq(d2)].dropna()
        if days1.size == 0 or days2.size == 0:
            continue
        pval = scipy.stats.ttest_ind(days1, days2).pvalue
        stats_results.append([d1, d2, days1.size, days2.size, pval])

    stats_results = pd.DataFrame(
        stats_results, 
        columns=["group1", "group2", "group1_size", "group2_size", "pval"]
    )
    stats_results['padj'] = statsmodels.stats.multitest.fdrcorrection(stats_results.pval)[1]
    stats_results['PC'] = pc
    stats_results['covariate'] = cat
    stats_results_sign = stats_results.loc[stats_results.padj.lt(0.05)]

    sns.boxplot(data=pc_df, x=cat, y=pc, ax=ax, color='lightgray')
    sns.swarmplot(data=pc_df, x=cat, y=pc, ax=ax, size=2, color='black')

    start_height = pc_df[pc].max()
    incrementer = 15 # px
    labels = [x.get_text() for x in ax.get_xticklabels()]
    q = ax.transData.inverted().transform([[0, 0], [0, incrementer]])
    y_offset = q[1][1] - q[0][1]
    gap = y_offset / 2
    y = start_height
    for _, r in stats_results_sign.iterrows():
        p = f'{r.padj:.3f}'
        
        # statistical annotation
        try:
            x1, x2 = labels.index(str(r.group1)), labels.index(str(r.group2))
        except:
            if type(r.group1) == float:
                group1 = int(r.group1)
            if type(r.group1) == float:
                group2 = int(r.group1)
            x1, x2 = labels.index(str(group1)), labels.index(str(group2))
        col = 'k'
        h = gap
        y += gap

        bracket = ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=0.5, c=col)
        txt = ax.text((x1+x2)*.5, y+.95*h, p, ha='center', va='bottom', color=col, size=6)
        y += y_offset
    
    ax.set_title(f'{cell_type.name}: {pc} vs {cat}', size=14)

    max_l = max(*[len(val) for val in pc_df[cat].cat.categories])
    if max_l > 15:
        trans = mpl.transforms.Affine2D().translate(6, 0)
        for t in ax.get_xticklabels():
            t.set_rotation(30)
            t.set_horizontalalignment("right")
            t.set_transform(t.get_transform() + trans)
    
    return fig, stats_results

In [31]:
%%time
all_pairwise = []
for cell_type in data:
    fig = cell_type.plot_pca_elbow()
    fig.savefig(cell_type.path / '_deseq2/pca_elbow.pdf')
    plt.close()
    weights = cell_type.get_pc_weights().round(5)
    weights.to_csv(cell_type.path / '_deseq2/pc_loadings.csv')
    pc_df = cell_type.get_pc_df(sample_meta)
    fig = plot_pc(cell_type, pc_df)
    fig.write_html(cell_type.path / '_deseq2/pc_plot.html')

    pc_names = pc_df.columns[pc_df.columns.str.startswith('PC')]
    pc_df = pc_df.set_index('sample').loc[:, pc_names]
    na_values = {col: 'NA' for col in categorical_covariates.columns}
    na_values.update(common_data.na_values)

    cat_covs = categorical_covariates.loc[pc_df.index].copy()
    for col in cat_covs.columns:
        cat_covs[col] = cat_covs[col].cat.remove_unused_categories()
    
    pc_cat_assoc = fi.test_association(
        pc_df,
        cat_covs,
        na_values
    )
    cg = plot_pc_cat_assoc(pc_cat_assoc)
    cg.fig.savefig(cell_type.path / '_deseq2/pc_cat_assoc.pdf')
    plt.close()

    pc_corrs, pc_corr_pvals = fi.test_correlation(
        pc_df,
        numerical_covariates.loc[pc_df.index]
    )
    cg = plot_pc_corrs(pc_corrs, pc_corr_pvals)
    cg.fig.savefig(cell_type.path / '_deseq2/pc_corrs.pdf')
    plt.close()

    pc_merge = pc_df.merge(cat_covs, left_index=True, right_index=True)
    stat_results = []
    for pc, cat in pc_cat_assoc.where(pc_cat_assoc < 0.05).stack().index.values:
        os.makedirs(cell_type.path / '_deseq2/pc_cat_assoc', exist_ok=True)

        fig, stats = plot_pc_cat_boxplot(cell_type.name, pc_merge, pc, cat, na_values)
        fig.savefig(cell_type.path / f'_deseq2/pc_cat_assoc/{pc}_{cat}.pdf')
        plt.close()
        stat_results.append(stats)
    if len(stat_results) > 0:
        stat_results = pd.concat(stat_results)
        stat_results.insert(0, 'cell_type', cell_type.name)
        all_pairwise.append(stat_results)
all_pairwise = pd.concat(all_pairwise)
all_pairwise.to_csv(BASE / '_pc_pairwise_covariate_tests.csv')

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7030: RuntimeWarning:

invalid value encountered in double_scalars

/projects/b1196/envs/serniczek/lib/python3.10/site-packages/scipy/stats/_axis_nan_

CPU times: user 4min 45s, sys: 2min 12s, total: 6min 58s
Wall time: 2min 58s


### Copy files to simpler folder structure for publishing

In [162]:
HTML = """
<!DOCTYPE html>
<html>
<head>
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <meta http-equiv="content-type" content="text/html; charset=utf8"/>
  <style type="text/css">
    html {
      margin: 0; padding: 0;
      font-size: 20px; font-family: Helvetica, Verdana, sans-serif;
    }
    body {margin: 0; padding: 50px 100px;}
    a {color: #1385cb}
    a:visited {color: #0e74bc}
    table {border-collapse: collapse; border-top: 1px solid #ccc; border-left: 1px solid #ccc;}
    table td, table th {padding: 2px 5px; border-bottom: 1px solid #ccc; border-right: 1px solid #ccc;}
    table td {font-size: 14px;}
  </style>
</head>
<body>
<table>
%s
</table>
</body>
</html>
"""

HTML_CELL_TYPE = """
<tr>
    <th>%s</th>
    <td><a href="%s" target="_blank">PC plot</a></td>
    <td><a href="%s" target="_blank">PC elbow plot</a></td>
    <td><a href="%s" target="_blank">PC loadings table</a></td>
    <td><a href="%s" target="_blank">PC categorical associations</a></td>
    <td><a href="%s" target="_blank">PC numerical associations</a></td>
</tr>
"""

In [163]:
def sanitize_name(name):
    return name.replace(' ', '_').replace('*', '').replace(';', '_and').replace('/', '_')

In [164]:
TARGET = common_data.DATA / '05_pseudobulk/10_pathogen-publish'
table = ''
for cell_type in data:
    dest = TARGET / cell_type.name
    os.makedirs(dest, exist_ok=True)
    shutil.copy2(cell_type.path / '_deseq2/pca_elbow.pdf', dest)
    shutil.copy2(cell_type.path / '_deseq2/pc_loadings.csv', dest / f'{sanitize_name(cell_type.name)}_pc_loadings.csv')
    shutil.copy2(cell_type.path / '_deseq2/pc_plot.html', dest)
    shutil.copy2(cell_type.path / '_deseq2/pc_cat_assoc.pdf', dest)
    shutil.copy2(cell_type.path / '_deseq2/pc_corrs.pdf', dest)
    if (cell_type.path / '_deseq2/pc_cat_assoc').exists():
        shutil.copytree(
            cell_type.path / '_deseq2/pc_cat_assoc', 
            dest / 'pc_cat_assoc', 
            dirs_exist_ok=True
        )
    cell_html = HTML_CELL_TYPE % (
        cell_type.name,
        cell_type.name + '/pc_plot.html',
        cell_type.name + '/pca_elbow.pdf',
        cell_type.name + f'/{sanitize_name(cell_type.name)}_pc_loadings.csv',
        cell_type.name + '/pc_cat_assoc.pdf',
        cell_type.name + '/pc_corrs.pdf',
    )
    table += cell_html
html = HTML % table
with open(TARGET / 'index.html', 'w') as out:
    out.write(html)